In [1]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# this version trains using the MNIST dataset, then tests on our own images
# (c) Tariq Rashid, 2016
# license is GPLv2

In [2]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [3]:
import os

In [4]:
# helper to load data from PNG image files
import imageio

In [5]:
# neural network class definition
class neuralNetwork2HiddenLayers:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodesLayer1, hiddennodesLayer2, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodesLayer1 = hiddennodesLayer1
        self.hnodesLayer2 = hiddennodesLayer2
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih1 = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodesLayer1, self.inodes))
        self.wh1h2 = numpy.random.normal(0.0, pow(self.hnodesLayer1, -0.5), (self.hnodesLayer2, self.hnodesLayer1))
        self.wh2o = numpy.random.normal(0.0, pow(self.hnodesLayer2, -0.5), (self.onodes, self.hnodesLayer2))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
   # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer 1
        hidden_inputs_l1 = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs_l1 = self.activation_function(hidden_inputs_l1)

        # calculate signals into hidden layer 2
        hidden_inputs_l2 = numpy.dot(self.wh1h2, hidden_outputs_l1)
        # calculate the signals emerging from hidden layer
        hidden_outputs_l2 = self.activation_function(hidden_inputs_l2)
 
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden_outputs_l2)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer 2 error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors_l2 = numpy.dot(self.wh2o.T, output_errors)
        # hidden layer 1 error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors_l1 = numpy.dot(self.wh1h2.T, hidden_errors_l2)
        
        # update the weights for the links between the hidden layer 2 and output layers
        self.wh2o += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs_l2))

        # update the weights for the links between the input and hidden layer l1
        self.wh1h2 += self.lr * numpy.dot((hidden_errors_l2 * hidden_outputs_l2 * (1.0 - hidden_outputs_l2)), numpy.transpose(hidden_outputs_l1))
        
        # update the weights for the links between the input and hidden layer l1
        self.wih1 += self.lr * numpy.dot((hidden_errors_l1 * hidden_outputs_l1 * (1.0 - hidden_outputs_l1)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer 1
        hidden_inputs_l1 = numpy.dot(self.wih1, inputs)

        print(hidden_inputs_l1)
        
        # calculate the signals emerging from hidden layer
        hidden_outputs_l1 = self.activation_function(hidden_inputs_l1)

        # calculate signals into hidden layer 2
        hidden_inputs_l2 = numpy.dot(self.wh1h2, hidden_outputs_l1)
        # calculate the signals emerging from hidden layer
        hidden_outputs_l2 = self.activation_function(hidden_inputs_l2)
 
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden_outputs_l2)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

    def loadWeights(self):
        self.wih1 = numpy.loadtxt("wih1.csv", delimiter=",")
        self.wh1h2 = numpy.loadtxt("wh1h2.csv", delimiter=",")
        self.wh2o = numpy.loadtxt("wh2o.csv", delimiter=",")
        pass    

In [6]:
# neural network class definition
class neuralNetwork1HiddenLayer:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        #dim1 = len(self.wih)
        #dim2 = len(self.wih[0])
        #print(dim1)
        #print(dim2)
        
        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)

        print(hidden_inputs)
        
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        print(hidden_outputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

    def loadWeights(self):
        self.wih = numpy.loadtxt("wih.csv", delimiter=",")
        self.who = numpy.loadtxt("who.csv", delimiter=",")
        pass

In [7]:
# number of input, hidden and output nodes
input_nodes = 9
hidden_nodes_l1 = 200
hidden_nodes_l2 = 9
output_nodes = 9

# learning rate
learning_rate = 0.1

# create instance of neural network
#n = neuralNetwork2HiddenLayers(input_nodes,hidden_nodes_l1,hidden_nodes_l2,output_nodes,learning_rate)
n = neuralNetwork1HiddenLayer(input_nodes,hidden_nodes_l1,output_nodes,learning_rate)

In [8]:
n.loadWeights()

In [9]:
# load the mnist training data CSV file into a list
training_data_file = open("nnTestData.txt", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [10]:
print("test the neural network")
# test the neural network

equals = 0
unequals = 0
i=0
for training_data in training_data_list:
    if i % 2 == 0:
        inputVector = numpy.asfarray(training_data.replace("\n", "").split(',')) 
    else:
        outputVector = numpy.asfarray(training_data.replace("\n", "").split(','))
        resultVector = n.query(inputVector)

        maxIdxOutputVector = numpy.argmax(outputVector)
        maxIdxResultVector = numpy.argmax(resultVector)

        if (maxIdxOutputVector == maxIdxResultVector):
            equals = equals + 1
        else:
            unequals = unequals + 1
    i=i+1
pass

print("Equals : " + str(equals))
print("Unequals : " + str(unequals))

test the neural network
[[-8.08291830e+00]
 [-5.17868167e+01]
 [-7.39953194e+00]
 [-4.55973924e+01]
 [-3.78161663e+01]
 [-4.20331579e+01]
 [-3.51399195e+01]
 [-5.49837357e+01]
 [-2.11946965e+00]
 [-5.81493138e+01]
 [-2.49985129e+01]
 [-7.58906718e+01]
 [-4.18565083e+00]
 [-4.93759174e+01]
 [-5.79425955e+01]
 [-5.96128966e+00]
 [-8.78544769e+01]
 [-2.36264627e+00]
 [-3.11883071e+01]
 [-1.67920867e+01]
 [ 1.67890894e+01]
 [-2.03331029e+01]
 [-7.34194770e+01]
 [-1.86742127e+01]
 [-4.26746950e+01]
 [ 1.27219141e+00]
 [-3.87959056e+01]
 [-3.81862850e+01]
 [ 2.65280455e+01]
 [-3.90075890e+01]
 [-4.04554238e+01]
 [-2.26321426e+01]
 [-5.84676391e+01]
 [-4.80328507e+00]
 [-6.27979687e+01]
 [-4.93125123e+01]
 [-4.76964627e+01]
 [-5.59226633e+01]
 [-1.95787341e+01]
 [-4.62214614e+01]
 [ 3.15861807e+01]
 [-1.46809355e+00]
 [-1.04216877e+01]
 [-3.15716172e+01]
 [-5.67373073e+01]
 [ 3.75183192e-01]
 [-5.61354992e+01]
 [-6.05161190e+01]
 [-9.62451092e+01]
 [-4.63082313e+01]
 [-5.29375711e+01]
 [-2.98